In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
import matplotlib.pyplot as plt

In [3]:
import sys
sys.argv = [""]

In [20]:
from cart_pole.utils import *
from cart_pole.agent import *
from torch.utils.tensorboard import SummaryWriter
from cart_pole.environment import *
import pdb

In [5]:
args = parse_args(seconds_since_epoch=1764177877)
writer = init_tracking(args)

In [22]:
envs = make_sync_vector_env(args)
agent = Agent(envs).to(args.device)
trainer = PPOTrainer(args, envs)
optimizer = optim.Adam(agent.parameters(),
                       lr=args.learning_rate,
                       eps=1e-5)

num_updates = args.total_timesteps // args.batch_size
trainer.rollout(agent, envs)

NameError: name 'copy' is not defined

In [ ]:
scaled_x = obs_scaler(storage.obs)

In [ ]:
pd.DataFrame(scaled_x.reshape((-1,) + (4,)).cpu()).describe()

In [ ]:
pd.DataFrame(storage.obs.reshape((-1,) + (4,)).cpu()).describe()